# Delta Lake Uniform

This notebook demonstrates Delta Lake Uniform, which allows you to read Delta tables with Iceberg reader clients. UniForm automatically generates Iceberg metadata asynchronously, without rewriting data, so that Iceberg clients can read Delta tables as if they were Iceberg tables.

In [1]:
from pathlib import Path

import delta
import pyspark
from delta import *
from pyspark.sql import functions as F

In [2]:
extras = ["io.delta:delta-iceberg_2.12:3.0.0rc1"]

builder = (
    pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
    .config("spark.jars.ivySettings", "../../ivy/3.0.0rc1.xml")
)

spark = configure_spark_with_delta_pip(builder, extra_packages=extras).getOrCreate()

:: loading settings :: file = ../../ivy/3.0.0rc1.xml
:: loading settings :: url = jar:file:/opt/homebrew/anaconda3/envs/pyspark-340-delta-300rc1/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/nick.karpov/.ivy2/cache
The jars for the packages stored in: /Users/nick.karpov/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
io.delta#delta-iceberg_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d154d83b-c551-48d1-b7c4-ba6c3a764f78;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.0.0rc1 in central
	found io.delta#delta-storage;3.0.0rc1 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found io.delta#delta-iceberg_2.12;3.0.0rc1 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.1.1 in central
	found com.github.ben-manes.caffeine#caffeine;2.9.3 in central
	found org.checkerframework#checker-qual;3.19.0 in central
	found com.google.errorprone#error_prone_annotations;2.10.0 in central
downloading https://repo1.maven.org/maven2/io/delta/delta-iceberg_2.12/3.0.0rc1/delta-iceberg_2.12-3.0.0rc1.jar ...
	[SUCCESSFUL ] io.delta#delta-iceberg_2.12;3.0.0rc1!delta-ice

In [7]:
# Create a Delta Table with Uniform enabled

spark.sql(
    """
    CREATE TABLE T(c1 BIGINT) 
    USING delta
    TBLPROPERTIES(
      'delta.universalFormat.enabledFormats' = 'iceberg')
    LOCATION '/tmp/tables/T'
"""
)

DataFrame[]

In [10]:
%%bash
# List the table directory
# Note the `metadata` folder which contains Iceberg metadata
ls -al /tmp/tables/T

total 0
drwxr-xr-x@ 4 nick.karpov  wheel  128 Aug  7 10:36 .
drwxr-xr-x@ 4 nick.karpov  wheel  128 Aug  7 10:36 ..
drwxr-xr-x@ 4 nick.karpov  wheel  128 Aug  7 10:36 _delta_log
drwxr-xr-x@ 6 nick.karpov  wheel  192 Aug  7 10:36 metadata


In [11]:
%%bash
# Take a closer look at the Iceberg metadata
ls -al /tmp/tables/T/metadata

total 32
drwxr-xr-x@ 6 nick.karpov  wheel   192 Aug  7 10:36 .
drwxr-xr-x@ 4 nick.karpov  wheel   128 Aug  7 10:36 ..
-rw-r--r--@ 1 nick.karpov  wheel    20 Aug  7 10:36 .v1.metadata.json.crc
-rw-r--r--@ 1 nick.karpov  wheel    12 Aug  7 10:36 .version-hint.text.crc
-rw-r--r--@ 1 nick.karpov  wheel  1043 Aug  7 10:36 v1.metadata.json
-rw-r--r--@ 1 nick.karpov  wheel     1 Aug  7 10:36 version-hint.text


In [12]:
# Append some new data

(
    spark.range(10).coalesce(1)
    .withColumnRenamed("id", "c1")
    .write.format("delta")
    .mode("append")
    .save("/tmp/tables/T")
)

In [22]:
%%bash
# List the data files. Only a single copy of the data files exists.
ls -p /tmp/tables/T | grep -v / 

part-00000-8e4ccae8-59e8-4dcd-959a-ec53cb7c7d0f-c000.snappy.parquet


In [23]:
%%bash
# List the Delta Lake metadata
ls -l /tmp/tables/T/_delta_log

total 16
-rw-r--r--@ 1 nick.karpov  wheel  1131 Aug  7 10:36 00000000000000000000.json
-rw-r--r--@ 1 nick.karpov  wheel   761 Aug  7 10:38 00000000000000000001.json


In [24]:
%%bash
# List the Iceberg metadata
ls -l /tmp/tables/T/metadata

total 48
-rw-r--r--@ 1 nick.karpov  wheel  5678 Aug  7 10:38 b985b02d-8f43-4c31-9fc0-dc1832a993ff-m0.avro
-rw-r--r--@ 1 nick.karpov  wheel  3742 Aug  7 10:38 snap-6163827902736455386-1-b985b02d-8f43-4c31-9fc0-dc1832a993ff.avro
-rw-r--r--@ 1 nick.karpov  wheel  1043 Aug  7 10:36 v1.metadata.json
-rw-r--r--@ 1 nick.karpov  wheel  1963 Aug  7 10:38 v2.metadata.json
-rw-r--r--@ 1 nick.karpov  wheel     1 Aug  7 10:38 version-hint.text
